导入需要的库

In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
%matplotlib inline

import itertools
import matplotlib.gridspec as gridspec
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
# from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
# from mlxtend.plotting import plot_learning_curves
# from mlxtend.plotting import plot_decision_regions

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

定义一个减小内存的函数

In [2]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

读取task3中将训练集和测试集一起进行特征工程得到的特征集

In [4]:
traintest=pd.read_csv('F:\\BaiduNetdiskDownload\\all_features.csv')

In [5]:
traintest

,Unnamed: 0,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,...,s_698,s_699,s_700,s_701,s_702,s_703,s_704,s_705,s_706,label
0,0,38.927945,0.660949,1.090709,0.848728,1.168685,0.982133,1.223496,1.236300,1.104172,...,-0.047438,0.554370,0.307586,0.564596,0.562960,0.591859,0.504124,0.528450,0.473568,0.0
1,1,19.445634,1.718217,1.280923,1.850706,1.460752,1.924501,1.925485,1.715938,2.079957,...,-0.109579,0.697446,0.398073,0.640969,0.270192,0.224925,0.645082,0.635135,0.297325,0.0
2,2,21.192974,1.814281,1.619051,1.215343,1.787166,2.146987,1.686190,1.540137,2.291031,...,-0.074042,0.321703,0.390386,0.716929,0.316524,0.422077,0.722742,0.680590,0.383754,2.0
3,3,42.113066,2.109550,0.619634,2.366413,2.071539,1.000340,2.728281,1.391727,2.017176,...,-0.184248,0.564669,0.623353,0.466980,0.651774,0.308915,0.550097,0.466904,0.494024,0.0
4,4,69.756786,0.194549,0.348882,0.092119,0.653924,0.231422,1.080003,0.711244,1.357904,...,0.070505,0.065835,0.051780,0.092940,0.103773,0.179405,-0.089611,0.091841,0.056867,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,119995,43.175130,1.776937,0.211527,1.986940,0.393550,1.693620,1.139395,1.459990,1.734535,...,-0.060254,0.507950,0.560192,0.541534,0.249750,0.608796,0.455444,0.535306,0.268471,-1.0
119996,119996,31.030782,1.451045,2.483726,1.105440,1.979721,2.821799,0.475276,2.782573,2.827882,...,0.016413,0.480380,0.459172,0.363756,0.427028,0.544692,0.754834,0.361866,0.536087,-1.0
119997,119997,31.648623,2.141301,0.546706,2.340499,1.362651,1.942634,2.043679,0.994065,2.248144,...,0.001012,0.768960,0.834159,0.672114,0.520215,0.341519,0.713419,0.664354,0.370047,-1.0
119998,119998,19.305442,0.221708,2.355288,1.051282,1.742370,2.164058,0.435583,2.649994,1.190594,...,-0.103574,0.521222,0.426435,0.636887,0.446365,0.551442,0.503703,0.635246,0.258394,-1.0


减小内存

In [8]:
traintest=reduce_mem_usage(traintest)

Memory usage of dataframe is 649.11 MB
Memory usage after optimization is: 162.51 MB
Decreased by 75.0%


In [9]:
traintest

,Unnamed: 0,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,...,s_698,s_699,s_700,s_701,s_702,s_703,s_704,s_705,s_706,label
0,0,38.937500,0.661133,1.090820,0.848633,1.168945,0.981934,1.223633,1.236328,1.104492,...,-0.047424,0.554199,0.307617,0.564453,0.562988,0.591797,0.503906,0.528320,0.473633,0.0
1,1,19.453125,1.717773,1.281250,1.850586,1.460938,1.924805,1.925781,1.715820,2.080078,...,-0.109558,0.697266,0.398193,0.641113,0.270264,0.224976,0.645020,0.635254,0.297363,0.0
2,2,21.187500,1.814453,1.619141,1.215820,1.787109,2.146484,1.686523,1.540039,2.291016,...,-0.074036,0.321777,0.390381,0.716797,0.316406,0.422119,0.722656,0.680664,0.383789,2.0
3,3,42.125000,2.109375,0.619629,2.367188,2.072266,1.000000,2.728516,1.391602,2.017578,...,-0.184204,0.564453,0.623535,0.467041,0.651855,0.308838,0.550293,0.466797,0.494141,0.0
4,4,69.750000,0.194580,0.348877,0.092102,0.653809,0.231445,1.080078,0.711426,1.357422,...,0.070496,0.065857,0.051788,0.092957,0.103760,0.179443,-0.089600,0.091858,0.056854,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,119995,43.187500,1.777344,0.211548,1.987305,0.393555,1.693359,1.139648,1.459961,1.734375,...,-0.060242,0.507812,0.560059,0.541504,0.249756,0.608887,0.455322,0.535156,0.268555,-1.0
119996,119996,31.031250,1.451172,2.484375,1.105469,1.979492,2.822266,0.475342,2.783203,2.828125,...,0.016418,0.480469,0.459229,0.363770,0.427002,0.544922,0.754883,0.361816,0.536133,-1.0
119997,119997,31.656250,2.140625,0.546875,2.339844,1.362305,1.942383,2.042969,0.994141,2.248047,...,0.001012,0.769043,0.833984,0.671875,0.520020,0.341553,0.713379,0.664551,0.370117,-1.0
119998,119998,19.312500,0.221680,2.355469,1.051758,1.742188,2.164062,0.435547,2.650391,1.190430,...,-0.103577,0.520996,0.426514,0.636719,0.446289,0.551270,0.503906,0.635254,0.258301,-1.0


删除索引列和标签列

In [12]:
data = traintest.drop(['Unnamed: 0','label'], axis=1)

In [13]:
data

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_697,s_698,s_699,s_700,s_701,s_702,s_703,s_704,s_705,s_706
0,38.937500,0.661133,1.090820,0.848633,1.168945,0.981934,1.223633,1.236328,1.104492,1.497070,...,0.531738,-0.047424,0.554199,0.307617,0.564453,0.562988,0.591797,0.503906,0.528320,0.473633
1,19.453125,1.717773,1.281250,1.850586,1.460938,1.924805,1.925781,1.715820,2.080078,1.818359,...,0.563477,-0.109558,0.697266,0.398193,0.641113,0.270264,0.224976,0.645020,0.635254,0.297363
2,21.187500,1.814453,1.619141,1.215820,1.787109,2.146484,1.686523,1.540039,2.291016,2.404297,...,0.712402,-0.074036,0.321777,0.390381,0.716797,0.316406,0.422119,0.722656,0.680664,0.383789
3,42.125000,2.109375,0.619629,2.367188,2.072266,1.000000,2.728516,1.391602,2.017578,2.611328,...,0.601562,-0.184204,0.564453,0.623535,0.467041,0.651855,0.308838,0.550293,0.466797,0.494141
4,69.750000,0.194580,0.348877,0.092102,0.653809,0.231445,1.080078,0.711426,1.357422,1.238281,...,0.015289,0.070496,0.065857,0.051788,0.092957,0.103760,0.179443,-0.089600,0.091858,0.056854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,43.187500,1.777344,0.211548,1.987305,0.393555,1.693359,1.139648,1.459961,1.734375,1.025391,...,0.546875,-0.060242,0.507812,0.560059,0.541504,0.249756,0.608887,0.455322,0.535156,0.268555
119996,31.031250,1.451172,2.484375,1.105469,1.979492,2.822266,0.475342,2.783203,2.828125,0.520020,...,0.491699,0.016418,0.480469,0.459229,0.363770,0.427002,0.544922,0.754883,0.361816,0.536133
119997,31.656250,2.140625,0.546875,2.339844,1.362305,1.942383,2.042969,0.994141,2.248047,1.006836,...,0.529785,0.001012,0.769043,0.833984,0.671875,0.520020,0.341553,0.713379,0.664551,0.370117
119998,19.312500,0.221680,2.355469,1.051758,1.742188,2.164062,0.435547,2.650391,1.190430,2.328125,...,0.527344,-0.103577,0.520996,0.426514,0.636719,0.446289,0.551270,0.503906,0.635254,0.258301


划分训练集

In [14]:
X_train = data.iloc[:100000,:]

训练集类别标签

In [18]:
y_train=traintest.iloc[0:100000,708]
y_train

0        0.0
1        0.0
2        2.0
3        0.0
4        2.0
        ... 
99995    0.0
99996    2.0
99997    3.0
99998    2.0
99999    0.0
Name: label, Length: 100000, dtype: float16

测试集

In [19]:
X_test=data.iloc[100000:,:]

将训练集划分为训练集和验证集，比例为4：1

In [21]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(X_train, y_train, test_size=0.2)

定义一些模型，随机森林，lgb，mlp神经网络

In [23]:
def build_model_rf(X_train,y_train):
    model = RandomForestRegressor(n_estimators = 100)
    model.fit(X_train, y_train)
    return model


def build_model_lgb(X_train,y_train):
    model = lgb.LGBMRegressor(num_leaves=63,learning_rate = 0.1,n_estimators = 100)
    model.fit(X_train, y_train)
    return model


def build_model_nn(X_train,y_train):
    model = MLPRegressor(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,solver='lbfgs')
    model.fit(X_train, y_train)
    return model

这里针对三个单模进行训练，其中subA_rf/lgb/nn都是可以提交的模型
单模没有进行调参，因此是弱分类器，效果可能不是很好。

In [ ]:
print('predict rf...')
model_rf = build_model_rf(X_train1,y_train1)
val_rf = model_rf.predict(X_val1)
subA_rf = model_rf.predict(X_test)


print('predict lgb...')
model_lgb = build_model_lgb(X_train1,y_train1)
val_lgb = model_lgb.predict(X_val1)
subA_lgb = model_rf.predict(X_test)


print('predict NN...')
model_nn = build_model_nn(X_train1,y_train1)
val_nn = model_nn.predict(X_val1)
subA_nn = model_rf.predict(X_test)

predict rf...


好像是因为给的特征太多，跑不动，此处放弃，令：可以根据RF的模型对特征重要性进行评估，筛选出相关系数大的特征，从而对特征进行降维处理，提升模型精度

fea_imp=model_rf.feature_importances_

加权融合
首先我们尝试加权融合模型：

如果没有给权重矩阵，就是均值融合模型
权重矩阵可以进行自定义，这里我们是用三个单模进行融合。如果有更多需要更改矩阵size

In [ ]:
# 加权融合模型，如果w没有变，就是均值融合
def Weighted_method(test_pre1,test_pre2,test_pre3,w=[1/3,1/3,1/3]):
    Weighted_result = w[0]*pd.Series(test_pre1)+w[1]*pd.Series(test_pre2)+w[2]*pd.Series(test_pre3)
    return Weighted_result

# 初始权重，可以进行自定义，这里我们随便设置一个权重
w = [0.2, 0.3, 0.5]

val_pre = Weighted_method(val_rf,val_lgb,val_nn,w)
MAE_Weighted = mean_absolute_error(y_val,val_pre)
print('MAE of Weighted of val:',MAE_Weighted)

In [ ]:
## 预测数据部分
subA = Weighted_method(subA_rf,subA_lgb,subA_nn,w)

## 生成提交文件
sub = pd.DataFrame()
sub['SaleID'] = X_test.index
sub['price'] = subA
sub.to_csv('./sub_Weighted.csv',index=False)

In [ ]:
train_rf_pred = model_rf.predict(X_train)
train_lgb_pred = model_lgb.predict(X_train)
train_nn_pred = model_nn.predict(X_train)

stacking_X_train = pd.DataFrame()
stacking_X_train['Method_1'] = train_rf_pred
stacking_X_train['Method_2'] = train_lgb_pred
stacking_X_train['Method_3'] = train_nn_pred

stacking_X_val = pd.DataFrame()
stacking_X_val['Method_1'] = val_rf
stacking_X_val['Method_2'] = val_lgb
stacking_X_val['Method_3'] = val_nn

stacking_X_test = pd.DataFrame()
stacking_X_test['Method_1'] = subA_rf
stacking_X_test['Method_2'] = subA_lgb
stacking_X_test['Method_3'] = subA_nn

In [ ]:
stacking_X_test.head()

In [ ]:
# 第二层是用random forest
model_lr_stacking = build_model_rf(stacking_X_train,y_train)

## 训练集
train_pre_Stacking = model_lr_stacking.predict(stacking_X_train)
print('MAE of stacking:',mean_absolute_error(y_train,train_pre_Stacking))

## 验证集
val_pre_Stacking = model_lr_stacking.predict(stacking_X_val)
print('MAE of stacking:',mean_absolute_error(y_val,val_pre_Stacking))

## 预测集
print('Predict stacking...')
subA_Stacking = model_lr_stacking.predict(stacking_X_test)

总结：task5主要提到了两种模型融合的方式，一种是多个单模型结果加权输出，另外一种是将多个模型的结果作为新的特征，然后带入新的模型训练，并的到最终预测模型，第一种模型权重占比比较重要，第二种的第二层模型容易造成过拟合，所以要采用比较简单的模型

由于特征太多，而sklearn不支持gpu，电脑跑不动，所以这里并没有继续运行下去，但是思路已经完全掌握，在尝试将这种模型导入自己研究方向的数据，收获良多呀